# Multiple Parallel Input and Multiple Step MLP Model

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


#### split a multivariate sequence into samples

In [2]:
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

## 1. Prepare data

#### Define input sequence

In [3]:
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [4]:
out_seq

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

#### convert to [rows, columns] structure

In [5]:
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [6]:
out_seq

array([[ 25],
       [ 45],
       [ 65],
       [ 85],
       [105],
       [125],
       [145],
       [165],
       [185]])

#### horizontally stack columns

In [7]:
dataset = np.hstack((in_seq1, in_seq2, out_seq))

#### choose a number of time steps

In [8]:
n_steps_in, n_steps_out = 3, 2

#### convert into input/output

In [9]:
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

In [10]:
for i in range(len(X)):
    print(X[i], y[i])

[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


#### flatten input

In [11]:
n_input = X.shape[1] * X.shape[2]
X = X.reshape((X.shape[0], n_input))

In [12]:
X

array([[ 10,  15,  25,  20,  25,  45,  30,  35,  65],
       [ 20,  25,  45,  30,  35,  65,  40,  45,  85],
       [ 30,  35,  65,  40,  45,  85,  50,  55, 105],
       [ 40,  45,  85,  50,  55, 105,  60,  65, 125],
       [ 50,  55, 105,  60,  65, 125,  70,  75, 145]])

#### flatten output

In [13]:
n_output = y.shape[1] * y.shape[2]
y = y.reshape((y.shape[0], n_output))

In [14]:
y

array([[ 40,  45,  85,  50,  55, 105],
       [ 50,  55, 105,  60,  65, 125],
       [ 60,  65, 125,  70,  75, 145],
       [ 70,  75, 145,  80,  85, 165],
       [ 80,  85, 165,  90,  95, 185]])

## 2. Define model

In [15]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mse')

## 3. Fit model

In [16]:
model.fit(X, y, epochs=2000, verbose=0)

## 3. Demonstrate prediction

In [17]:
x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[ 90.65106   95.55715  187.9247   102.07478  107.687386 208.862   ]]
